In [1]:
pip install laion-clap

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 34.6 MB/s eta 0:00:00
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12065 sha256=a90bf60199d76fb753406d52d535ace89ee03bdbf6e36154afde4cb334ca6918
  Stored in directory: /root/.cache/pip/wheels/8d/bb/b2/5353b966ac6f3c5e1000629a9a5f6aed41794487f551e32efc
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=3b99c403783da4a46e5c58fc1351650bfad08d66709c4f6791202d20265eaa15
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built progressbar wget
  Attempting uninstall: huggingface-hub
    

In [2]:
import json
import logging
import os
import pandas as pd

from laion_clap import CLAP_Module
import torch
import torch.hub

logger = logging.getLogger(__name__)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

In [3]:
import torch
import torch.nn.functional as F


def average_embeddings(
    embeddings: torch.Tensor, normalize: bool = True
) -> torch.Tensor:
    """
    Compute the averaged embedding of a set of embeddings using best practices.
    """
    if embeddings.ndim == 3 and embeddings.shape[1] == 1:
        embeddings = embeddings.squeeze(1)  # Convert (N, 1, D) → (N, D)
    elif embeddings.ndim != 2:
        raise ValueError(f"Expected embeddings of shape (N, D), got {embeddings.shape}")

    if normalize:
        embeddings = F.normalize(embeddings, dim=-1)

    mean_emb = embeddings.mean(dim=0)

    if normalize:
        mean_emb = F.normalize(mean_emb, dim=-1)

    return mean_emb

In [4]:
_CLAP_MODEL = None


def clap_model():
    global _CLAP_MODEL
    if _CLAP_MODEL is not None:
        return _CLAP_MODEL

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = CLAP_Module(enable_fusion=False)

    weights_url = "https://huggingface.co/lukewys/laion_clap/resolve/main/music_audioset_epoch_15_esc_90.14.pt"

    try:
        state_dict = torch.hub.load_state_dict_from_url(
            weights_url, map_location=device, weights_only=False
        )
        model.load_state_dict(state_dict, strict=False)
    except Exception as e:
        logger.error(f"Error loading model weights from {weights_url}: {e}")
        raise

    model.to(device)
    model.eval()

    logger.info(f"Modelo CLAP cargado correctamente en {device}")
    _CLAP_MODEL = model
    return model


def get_text_embeddings_in_batches(descriptors: list[str], batch_size: int = 16):
    model = clap_model()
    embeddings = []
    with torch.no_grad():
        for pos in range(0, len(descriptors), batch_size):
            batch = descriptors[pos : pos + batch_size]
            emb = F.normalize(
                model.get_text_embedding(batch, use_tensor=True), p=2, dim=-1
            )
            for text, embedding in zip(batch, emb.cpu()):
                embeddings.append({"text": text, "embedding": embedding.tolist()})

    return embeddings


def concat_salient_words(word_pos_map: dict[str, str]) -> str:
    return " ".join(word_pos_map.keys())


def load_spanio_captions() -> list[str]:
    file_path = "/kaggle/input/spanio/spanio_captions.json"
    with open(file_path, "r") as f:
        captions = [val for key, val in json.load(f).items()]

    return [concat_salient_words(caption) for caption in captions]

In [15]:
import csv


def load_only_audio_captions() -> list[str]:
    file_path = "/kaggle/input/audiocaps/only_audio_caps.csv"
    caps: list[str] = []

    with open(file_path, "r") as f:
        csv_reader = csv.reader(f)
        caps = [row[0] for row in csv_reader]

    return caps

In [20]:
captions_ = load_spanio_captions()
caption_embeddings_map = get_text_embeddings_in_batches(captions_)

pd.DataFrame(caption_embeddings_map).to_csv(
    "spanio_captions_embeddings.csv", index=False
)

In [19]:
audio_caps = load_only_audio_captions()

audio_caps_map = get_text_embeddings_in_batches(audio_caps)
pd.DataFrame(audio_caps_map).to_csv("audio_caps_embeddings.csv", index=False)

In [11]:
model = clap_model()

sent_emb = model.get_text_embedding(["sweet"], use_tensor=True)
sent_emb = F.normalize(sent_emb, dim=-1)

print(torch.norm(sent_emb, dim=-1))
emb = sent_emb.cpu().tolist()
print(emb)

pd.DataFrame(
    [
        emb,
    ]
).to_csv("simple_embeddings.csv")

tensor([1.], device='cuda:0', grad_fn=<LinalgVectorNormBackward0>)
[[-0.04226687550544739, 0.043931882828474045, -0.03497587889432907, -0.07991821318864822, -0.02514909766614437, -0.04352453723549843, 0.04772472381591797, 0.008386715315282345, -0.03981844335794449, -0.01233937032520771, 0.015822062268853188, -0.010329811833798885, -0.05462081357836723, -0.009628348983824253, 0.07402922958135605, 0.022557662799954414, -0.06276911497116089, -0.046550288796424866, -0.029799574986100197, 0.012104989029467106, 0.010932580567896366, 0.03845226764678955, -0.0008145154570229352, -0.011602257378399372, 0.06695506721735, 0.0196172334253788, 0.022638583555817604, -0.03012833371758461, 0.04452405497431755, -0.000682803918607533, 0.08728726208209991, 0.01573316939175129, -0.058674298226833344, -0.07345812767744064, 0.03934356942772865, 0.002483195159584284, -0.03970842435956001, -0.04915536567568779, 0.009294159710407257, -0.0019443753408268094, 0.04773446172475815, -0.040968600660562515, 0.0032566

In [ ]:
# model = clap_model()

# emb_list = [
#     # model.get_text_embedding(["sweet"], use_tensor=True),
#     # model.get_text_embedding(["melancholic"], use_tensor=True),
#     # model.get_text_embedding(["harp"], use_tensor=True),
#     # model.get_text_embedding(["sweet"], use_tensor=True),
#     # model.get_text_embedding(["piece"], use_tensor=True),
# ]

# # Stack into a single tensor
# embeddings = torch.stack(emb_list)  # shape (4, D)

# # Compute averaged embedding
# avg_emb = average_embeddings(embeddings)

# # Cosine similarity between the averaged and sentence embeddings
# similarity = F.cosine_similarity(avg_emb, sent_emb).item()
# print(f"Cosine similarity: {similarity:.4f}")